#### LeetCode - SQL - #608

Refer [608. Tree Node](https://leetcode.com/problems/tree-node/description/)

Each node in the tree can be one of three types:

-   **"Leaf"**: if the node is a leaf node.
-   **"Root"**: if the node is the root of the tree.
-   **"Inner"**: If the node is neither a leaf node nor a root node.

Write a solution to report the type of each node in the tree.

Return the result table in  **any order**.

In [56]:
data_tree = [[1, None], [2, 1], [3, 1], [4, 2], [5, 2]]
columns_tree = ['id', 'p_id']
schema_tree = {'id':'Int64', 'p_id':'Int64'}

In [57]:
import pandas

pandas_df_tree = pandas.DataFrame(data=data_tree, columns=columns_tree).astype(schema_tree)

In [ ]:
from pyspark.sql import SparkSession

spark_context = SparkSession.builder.appName('LeetCode SQL').getOrCreate()

# Spark DataFrame using Pandas DataFrame
# Creating Spark Dataframe from pandas dataframe converts 'None' to 'NaN'
df_tree = spark_context.createDataFrame(pandas_df_tree)
df_tree.printSchema()
df_tree.show()

In [ ]:
# Creating Spark Dataframe directly in PySpark function converts 'None' to 'Null'
df_tree = spark_context.createDataFrame(data_tree, schema="id integer, p_id integer")
df_tree.printSchema()
df_tree.show()

In [60]:
from pyspark.sql import functions as F

df_tree_p_id = df_tree.filter(df_tree.p_id.isNotNull())
list_tree_p_id = list(df_tree_p_id.toPandas()["p_id"])

df_tree \
    .select(
        df_tree.id.alias("id"),
        F.when(df_tree.p_id.isNull(), 'Root').otherwise(
            F.when(df_tree.id.isin(list_tree_p_id), 'Inner').otherwise(
                'Leaf'
            )
        ).alias('type')
    ) \
    .show()

+---+-----+
| id| type|
+---+-----+
|  1| Root|
|  2|Inner|
|  3| Leaf|
|  4| Leaf|
|  5| Leaf|
+---+-----+


#### Using Join

In [61]:
df_tree_parent = df_tree.alias('parent')
df_tree_child = df_tree.alias('child')

df_tree_joined \
    = df_tree_parent \
        .join(df_tree_child, F.col("parent.id") == F.col("child.p_id"), "left") \
        .select(
            F.col("parent.id").alias("parent_id"),
            F.col("parent.p_id").alias("parent_p_id"),
            F.col("child.id").alias("child_id")
        )   

df_tree_joined \
    .select(
        df_tree_joined.parent_id.alias("id"),
        F.when(df_tree_joined.parent_p_id.isNull(), 'Root').otherwise(
            F.when(df_tree_joined.child_id.isNull(), 'Leaf').otherwise(
                'Inner'
            )
        ).alias('type')
    ).distinct() \
    .show()

+---+-----+
| id| type|
+---+-----+
|  1| Root|
|  2|Inner|
|  3| Leaf|
|  4| Leaf|
|  5| Leaf|
+---+-----+


#### Using Join and groupBy

In [69]:
df_tree_joined \
    .groupBy(df_tree_joined.parent_id.alias("id")) \
    .agg(
        F.when(F.first(df_tree_joined.parent_p_id).isNull(), 'Root').otherwise(
            F.when(F.first(df_tree_joined.child_id).isNull(), 'Leaf').otherwise(
                'Inner'
            )
        ).alias('type')
    ) \
    .show()

+---+-----+
| id| type|
+---+-----+
|  1| Root|
|  2|Inner|
|  3| Leaf|
|  4| Leaf|
|  5| Leaf|
+---+-----+
